In [0]:
#export
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
import fastai
import torch.nn as nn

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [0]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
x_train, y_train, x_valid, y_valid = get_data()
train_mean, train_std = x_train.mean(), x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [5]:
x_train = x_train.view(-1, 1, 28, 28)
x_valid = x_valid.view(-1, 1, 28, 28)
x_train.shape, x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [6]:
n,*_ = x_train.shape
c = y_train.max()+1
nh=32
n,c

(50000, tensor(10))

In [0]:
l1 = nn.Conv2d(1, nh, 5)

In [0]:
x = x_valid[:100]

In [11]:
x.shape

torch.Size([100, 1, 28, 28])

In [0]:
def stats(x):
    return x.mean(), x.std()

In [13]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [15]:
stats(l1.weight), stats(l1.bias)

((tensor(0.0031, grad_fn=<MeanBackward0>),
  tensor(0.1146, grad_fn=<StdBackward0>)),
 (tensor(0.0164, grad_fn=<MeanBackward0>),
  tensor(0.1250, grad_fn=<StdBackward0>)))

In [0]:
t = l1(x)

In [17]:
stats(t)

(tensor(0.0239, grad_fn=<MeanBackward0>),
 tensor(0.6324, grad_fn=<StdBackward0>))

In [18]:
nn.init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0218, grad_fn=<MeanBackward0>),
 tensor(0.9674, grad_fn=<StdBackward0>))

In [0]:
import torch.nn.functional as F

In [0]:
def f1(x, a=0): 
    return F.leaky_relu(l1(x), a)

In [26]:
nn.init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5013, grad_fn=<MeanBackward0>),
 tensor(0.9060, grad_fn=<StdBackward0>))

In [27]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.1908, grad_fn=<MeanBackward0>),
 tensor(0.4039, grad_fn=<StdBackward0>))

In [28]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [30]:
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [31]:
nf, ni, *_ = l1.weight.shape
nf, ni

(32, 1)

In [32]:
fan_in = ni*rec_fs
fan_out = nf*rec_fs
fan_in, fan_out

(25, 800)

In [0]:
def gain(a): 
    return math.sqrt(2.0 / (1 + a**2))

In [34]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [35]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5807)

In [36]:
1/math.sqrt(3.)

0.5773502691896258

In [0]:
def kaiming2(x, a, use_fan_out=False):
    nf, ni, *_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [38]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.4341, grad_fn=<MeanBackward0>),
 tensor(0.8354, grad_fn=<StdBackward0>))

In [39]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2160, grad_fn=<MeanBackward0>),
 tensor(0.4130, grad_fn=<StdBackward0>))

In [0]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [0]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [0]:
y = y_valid[:100].float()

In [43]:
t = m(x)
stats(t)

(tensor(-0.0604, grad_fn=<MeanBackward0>),
 tensor(0.0096, grad_fn=<StdBackward0>))

In [0]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [0]:
l = mse(t,y)
l.backward()

In [47]:
stats(m[0].weight.grad)

(tensor(-0.0080), tensor(0.0539))

In [0]:
for l in m:
    if isinstance(l,nn.Conv2d):
        nn.init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [50]:

t = m(x)
stats(t)

(tensor(-0.4967, grad_fn=<MeanBackward0>),
 tensor(0.1707, grad_fn=<StdBackward0>))

In [51]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0146), tensor(0.4913))